In [265]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns

In [241]:
train = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/train.csv')

In [250]:
train1 = train.drop(['locationID_x.1','locationID_y.1','categoryID_x.1','categoryID_y.1'],axis = 1)

In [251]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2991396 entries, 0 to 2991395
Data columns (total 31 columns):
Unnamed: 0            int64
itemID_1              int64
itemID_2              int64
isDuplicate           int64
generationMethod      int64
itemID_x              int64
categoryID_x          int64
title_x               object
description_x         object
images_array_x        object
attrsJSON_x           object
price_x               float64
locationID_x          int64
metroID_x             float64
lat_x                 float64
lon_x                 float64
itemID_y              int64
categoryID_y          int64
title_y               object
description_y         object
images_array_y        object
attrsJSON_y           object
price_y               float64
locationID_y          int64
metroID_y             float64
lat_y                 float64
lon_y                 float64
regionID_x            int64
regionID_y            int64
parentCategoryID_x    int64
parentCategoryID_y    i

Classification Models: LDA, QDA, Logistic regression, SVM, Decision Trees, Random Forests, various forms of Boosting

# 1 Tree based methods

## 1.1 Decision Tree

In [254]:
#metroID有太多 missing data
#title, attrsJSON, description 不是英文
#images 暂时不用
#item id 重复了
train2 = train1.drop(['Unnamed: 0', 'title_x','title_y', 'description_x', 'description_y', 'images_array_x',
                      'images_array_y', 'attrsJSON_x', 'attrsJSON_y', 'generationMethod', 'metroID_x', 
                      'metroID_y','itemID_x', 'itemID_y'], axis=1)

In [255]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2991396 entries, 0 to 2991395
Data columns (total 17 columns):
itemID_1              int64
itemID_2              int64
isDuplicate           int64
categoryID_x          int64
price_x               float64
locationID_x          int64
lat_x                 float64
lon_x                 float64
categoryID_y          int64
price_y               float64
locationID_y          int64
lat_y                 float64
lon_y                 float64
regionID_x            int64
regionID_y            int64
parentCategoryID_x    int64
parentCategoryID_y    int64
dtypes: float64(6), int64(11)
memory usage: 388.0 MB


In [258]:
train3 = train2.dropna()
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2652009 entries, 0 to 2991395
Data columns (total 17 columns):
itemID_1              int64
itemID_2              int64
isDuplicate           int64
categoryID_x          int64
price_x               float64
locationID_x          int64
lat_x                 float64
lon_x                 float64
categoryID_y          int64
price_y               float64
locationID_y          int64
lat_y                 float64
lon_y                 float64
regionID_x            int64
regionID_y            int64
parentCategoryID_x    int64
parentCategoryID_y    int64
dtypes: float64(6), int64(11)
memory usage: 364.2 MB


In [261]:
train3.head(10)

,itemID_1,itemID_2,isDuplicate,categoryID_x,price_x,locationID_x,lat_x,lon_x,categoryID_y,price_y,locationID_y,lat_y,lon_y,regionID_x,regionID_y,parentCategoryID_x,parentCategoryID_y
0,1,4112648,1,81,300000.0,648140,64.686946,30.815924,81,300000.0,648140,64.686946,30.815924,648070,648070,1,1
1,3,1991275,1,14,300000.0,639040,55.678037,37.256548,14,330000.0,639040,55.678037,37.256548,637680,637680,1,1
2,4,1223296,0,84,3500.0,640650,56.239398,43.460458,84,3500.0,640650,56.239398,43.460458,640310,640310,6,6
3,7,1058851,1,84,13500.0,662210,55.777170,37.586194,84,13500.0,662210,56.135459,47.235484,662000,662000,6,6
4,8,2161930,1,39,500.0,624360,55.777170,37.586194,39,600.0,624360,55.777170,37.586194,624300,624300,7,7
5,9,694103,1,39,7000.0,644200,58.004785,56.237654,39,7000.0,644200,58.004785,56.237654,643700,643700,7,7
6,12,5637025,0,9,445000.0,631060,44.219841,42.058825,9,450000.0,631060,44.219841,42.058825,630750,630750,1,1
7,12,5279740,0,9,445000.0,631060,44.219841,42.058825,9,455000.0,631060,44.219841,42.058825,630750,630750,1,1
8,15,113701,0,32,1600.0,662810,57.622434,39.887894,32,5000.0,662810,57.622434,39.887894,662530,662530,6,6
9,16,2467698,0,27,1000.0,657600,56.495116,84.972128,27,7500.0,657600,56.495116,84.972128,657310,657310,5,5


In [280]:
#如何两个商品是一样的，那么price,location,lat,lon,category,region,parentcategory
#所以相减 看一下结果是不是0

y = train3.isDuplicate

price = abs(train3.price_x - train3.price_y)
location = abs(train3.locationID_x - train3.locationID_y)
lat = abs(train3.lat_x - train3.lat_y)
lon = abs(train3.lon_x - train3.lon_y)
category = abs(train3.categoryID_x - train3.categoryID_y)
region = abs(train3.regionID_x - train3.regionID_y)
parentCategory = abs(train3.parentCategoryID_x - train3.parentCategoryID_y)

d = {'price': price, 'location': location, 'lat': lat, 'lon': lon, 
     'category': category, 'region': region, 'parentCategory': parentCategory}
X = pd.DataFrame(data=d)

In [281]:
X.head()

,price,location,lat,lon,category,region,parentCategory
0,0.0,0,0.000000,0.00000,0,0,0
1,30000.0,0,0.000000,0.00000,0,0,0
2,0.0,0,0.000000,0.00000,0,0,0
3,0.0,0,0.358289,9.64929,0,0,0
4,100.0,0,0.000000,0.00000,0,0,0


In [282]:
y = train3[['isDuplicate']]

In [421]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
model = dtree.fit(X_train,y_train)

In [414]:
prediction = model.predict(X_test)

In [415]:
prediction

array([0, 0, 1, ..., 0, 0, 0])

In [416]:
from sklearn.metrics import classification_report, confusion_matrix

In [417]:
print(confusion_matrix(y_test, prediction))

[[387019 126972]
 [134554 226618]]


In [418]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.74      0.75      0.75    513991
           1       0.64      0.63      0.63    361172

   micro avg       0.70      0.70      0.70    875163
   macro avg       0.69      0.69      0.69    875163
weighted avg       0.70      0.70      0.70    875163



We have a pretty good prediction already for the decision tree with high precision.


## 1.2 Random Forest

In [428]:
from sklearn.ensemble import RandomForestClassifier

In [429]:
rfc = RandomForestClassifier(n_estimators = 50)

In [430]:
rfc.fit(X_train, y_train)

/Users/jerry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [431]:
rfc_pred = rfc.predict(X_test)

In [432]:
print(confusion_matrix(y_test, rfc_pred))

[[386832 126842]
 [130901 230588]]


In [433]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75    513674
           1       0.65      0.64      0.64    361489

   micro avg       0.71      0.71      0.71    875163
   macro avg       0.70      0.70      0.70    875163
weighted avg       0.71      0.71      0.71    875163



There is no huge improvement over the simple decision tree but random forest is still a bit better. 

In [434]:
rfc_pred_prob = rfc.predict_proba(X_test)[:,1]

In [435]:
import numpy as np
from sklearn import metrics

In [436]:
y_test1 = np.array(y_test['isDuplicate'])
y_test1

array([0, 0, 0, ..., 0, 1, 0])

In [437]:
rfc_pred_prob1 = np.array(rfc_pred_prob)
rfc_pred_prob1

array([0.30790541, 0.31410889, 0.60755908, ..., 0.60755908, 0.59206476,
       0.42100503])

In [438]:
fpr, tpr, thresholds = metrics.roc_curve(y_test1, rfc_pred_prob1)
metrics.auc(fpr, tpr)

0.7630777237365813

# Test Data

In [439]:
test = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/test.csv')

In [440]:
test1 = test.drop(['locationID_x.1','locationID_y.1','categoryID_x.1','categoryID_y.1'],axis = 1)

In [443]:
test2 = test1.drop(['Unnamed: 0', 'title_x','title_y', 'description_x', 'description_y', 'images_array_x',
                      'images_array_y', 'attrsJSON_x', 'attrsJSON_y', 'metroID_x', 
                      'metroID_y','itemID_x', 'itemID_y'], axis=1)

In [472]:
test2.head()

,id,itemID_1,itemID_2,categoryID_x,price_x,locationID_x,lat_x,lon_x,categoryID_y,price_y,locationID_y,lat_y,lon_y,regionID_x,regionID_y,parentCategoryID_x,parentCategoryID_y
0,0,5,4670875,115,NaN,637640,55.760211,37.577211,115,NaN,637640,55.760211,37.577211,637640,637640,113,113
1,1,5,787210,115,NaN,637640,55.760211,37.577211,115,NaN,637640,55.760211,37.577211,637640,637640,113,113
2,2,6,1705280,10,NaN,637160,59.645846,33.513035,10,1500.0,637160,59.645846,33.513035,636370,636370,1,1
3,3,11,3020777,101,1300.0,650400,55.817608,49.097646,101,490.0,650400,55.817608,49.097646,650130,650130,6,6
4,4,23,5316130,27,15000.0,637640,55.827080,37.437604,27,15000.0,637640,55.827080,37.437604,637640,637640,5,5


In [459]:
#如何两个商品是一样的，那么price,location,lat,lon,category,region,parentcategory
#所以相减 看一下结果是不是0


price1 = abs(test2.price_x - test2.price_y)
location1 = abs(test2.locationID_x - test2.locationID_y)
lat1 = abs(test2.lat_x - test2.lat_y)
lon1 = abs(test2.lon_x - test2.lon_y)
category1 = abs(test2.categoryID_x - test2.categoryID_y)
region1 = abs(test2.regionID_x - test2.regionID_y)
parentCategory1 = abs(test2.parentCategoryID_x - test2.parentCategoryID_y)

d1 = {'price': price1, 'location': location1, 'lat': lat1, 'lon': lon1, 
     'category': category1, 'region': region1, 'parentCategory': parentCategory1}
X1 = pd.DataFrame(data=d1)

In [461]:
X1.head()

,price,location,lat,lon,category,region,parentCategory
0,NaN,0,0.0,0.0,0,0,0
1,NaN,0,0.0,0.0,0,0,0
2,NaN,0,0.0,0.0,0,0,0
3,810.0,0,0.0,0.0,0,0,0
4,0.0,0,0.0,0.0,0,0,0


In [462]:
X1.isna().sum() #price有很多nan，需要

price             154247
location               0
lat                    0
lon                    0
category               0
region                 0
parentCategory         0
dtype: int64

In [463]:
list(X1.columns)

['price', 'location', 'lat', 'lon', 'category', 'region', 'parentCategory']

## Imputation

In [447]:
from sklearn.impute import SimpleImputer
import numpy as np

In [464]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X1)
X2 = imp_mean.transform(X1)

In [466]:
X2.shape

(1044196, 7)

In [468]:
X3 = pd.DataFrame(data = X2, columns=list(X1.columns))

In [469]:
X3.head()

,price,location,lat,lon,category,region,parentCategory
0,3.406927e+07,0.0,0.0,0.0,0.0,0.0,0.0
1,3.406927e+07,0.0,0.0,0.0,0.0,0.0,0.0
2,3.406927e+07,0.0,0.0,0.0,0.0,0.0,0.0
3,8.100000e+02,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0


## Prediction

In [470]:
pred_test = rfc.predict_proba(X3)[:,1]
pred_test = np.array(pred_test)

In [471]:
pred_test

array([0.        , 0.        , 0.        , ..., 0.45348892, 0.1702134 ,
       0.45348892])

In [473]:
index = np.array(test2['id'].astype(np.int))

In [474]:
index

array([      0,       1,       2, ..., 1044193, 1044194, 1044195])

In [475]:
pred_test1 = {'id':index,'probability':pred_test}

In [476]:
pred_test2 = pd.DataFrame(pred_test1, columns =['id','probability'])

In [477]:
pred_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044196 entries, 0 to 1044195
Data columns (total 2 columns):
id             1044196 non-null int64
probability    1044196 non-null float64
dtypes: float64(1), int64(1)
memory usage: 15.9 MB


In [478]:
pred_test2.to_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/prediction.csv',index=False)